In [1]:
import numpy as np
import random
import math

from matplotlib import pyplot as plt

In [7]:
# Paramètres
n = m = 18 # nombre de lignes = nombre de colonnes
initial_size = 2 # taille de la fourmilière initiale
max_food_quantity = 500 #quantité maximale de nourriture par source de nourriture
neigh = 1 #voisinage / champ de vision de chaque fourmi

In [10]:
def generate_simple_map(n, m): # n : lignes m : colonnes
    """Préconditions : m > 0 et n > 0, initial_size > 0
    Renvoie un monde de taille n*m avec une fourmilière au centre et un point de nourriture en périphérie"""
    map = np.array([[" " for i in range(0, n)] for j in range(0, n)]) # on crée la matrice vide (avec des " " partout)
    # on crée la fourmilière au centre
    for i in range(0, initial_size):
        for k in range(0, initial_size):
            map = np.delete(map, m*(n//2 - initial_size//2 + i) + m//2 - initial_size//2 + k) #on copie le monde en enlevant une à une les cases du carré de taille initial_size situé au centre
            map = np.insert(map, m*(n//2 - initial_size//2 + i) + m//2 - initial_size//2 + k, "h") #on remplace les cases initialement vide (avec un " ") par une case "fourmilière" (avec un "h" pour "home")
            map = np.array([[map[m*j + i] for i in range(0, m)] for j in range(0, n) ])
    
    # on crée un point de nourriture
    food_line = random.randint(0, 2*n//5 - 1) # on définit ainsi la périphérie comme étant les cases du bord de la matrice avec un largeur de n/5
    food_column = random.randint(0, 2*m//5 - 1)
    if food_line > n//5 - 1: # on est sur la périphérie du bas
        food_line = n - 2 - n//5 + food_line
    if food_column > m//5 - 1: # on est sur la périphérie de droite
        food_column = m - 2 - m//5 + food_column
    where = random.sample([food_line, food_column], 1) # on définit une périphérie où se mettre (horizontale ou verticale)
    if where == food_line: # on est sur une périphérie en haut ou en bas
        food_column = random.randint(0, m - 1) # on définit une case au hasard sur cette périphérie
    else : #on est sur une périphérie latérale
        food_line = random.randint(0, n - 1) # on définit une case au hasard sur cette périphérie
    # on place dans la matrice de point de nourriture de quantité aléatoire food_quantity et de position périphérique aléatoire (food_line, food_column)
    food_quantity = random.randint(1, max_food_quantity) # on attribue à ce point de nourriture une certaine quantité de nourriture
    print(food_quantity)
    map = np.delete(map, m*food_line + food_column)
    map = np.insert(map, m*food_line + food_column, food_quantity)
    map = np.array([[map[m*j + i] for i in range(0, m)] for j in range(0, n) ])       
    
    return map
print (generate_simple_map(n, m))

491
[[' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 ['4' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'h' 'h' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'h' 'h' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' '

In [76]:
map = generate_simple_map(n, m)
print(map)

def spatial_neighborhood (map, a, b, neigh): #world : List[List[int]], a : int(ligne), b : int(colonne), neigh : int
    
    """Préconditions :0 <= a <= n-1  and 0 <= b <= m-1 and n >= 2*neigh and m >= 2*neigh
    Retourne la matrice des types des voisins de de l'individu en [a, b] avec a la ligne et b la colonne"""
    
    voisin = np.array([[" " for i in range(0, 2*neigh + 1)] for j in range(0, 2*neigh + 1)]) #grille vide de la taille du voisinage
    
    world_without_ind = np.delete(map, m*a + b) #on copie le monde sans l'individu
    world_without_ind = np.insert(world_without_ind, m*a + b, " ") #on remplace la place de l'individu par une case vide (avec un " ")
    world_without_ind = np.array([[world_without_ind[m*j + i] for i in range(0, m)] for j in range(0, n) ]) #on remet le monde sous forme de tableau
    

    if a - neigh >= 0 and a + neigh <= n - 1: # on teste les lignes, il n'y a pas de problème avec les bords (n-1 car les indices commencent à 0)
        if b - neigh >= 0 and b + neigh <= m - 1: #on teste les colonnes, il n'y a pas de problèmes avec les bords
            voisin = np.array([[world_without_ind[a-neigh + j, b-neigh + i] for i in range(0, 2*neigh + 1)] for j in range(0, 2*neigh + 1)]) 
        elif b - neigh < 0: # on atteint le bord gauche de la matrice
            voisin = np.array([[world_without_ind[a - neigh + j, i] for i in range(0, b + neigh + 1)] for j in range(0, 2*neigh + 1)])
        elif b + neigh > m - 1: # on atteint le bord droit
            voisin = np.array([[world_without_ind[a - neigh + j, i] for i in range(b - neigh, m)] for j in range(0, 2*neigh + 1)])
    elif a - neigh < 0 : # on atteint le bord haut
        if b - neigh >= 0 and b + neigh <= m - 1: #on teste les colonnes, il n'y a pas de problèmes avec les bords
            voisin = np.array([[world_without_ind[j, b-neigh + i] for i in range(0, 2*neigh + 1)] for j in range(0, a + neigh + 1)])
        elif b - neigh < 0: # on atteint le bord gauche de la matrice
            voisin = np.array([[world_without_ind[j, i] for i in range(0, b + neigh + 1)] for j in range(0, a + neigh + 1)])
        elif b + neigh > m - 1: # on atteint le bord droit
            voisin = np.array([[world_without_ind[j, i] for i in range(b - neigh, m)] for j in range(0, a + neigh + 1)])
    else : # a + neigh > n - 1
        if b - neigh >= 0 and b + neigh <= m - 1: #on teste les colonnes, il n'y a pas de problèmes avec les bords
            voisin = np.array([[world_without_ind[j, b-neigh + i] for i in range(0, 2*neigh + 1)] for j in range(a - neigh, n)]) #on copie 
        elif b - neigh < 0: # on atteint le bord gauche de la matrice
            voisin = np.array([[world_without_ind[j, i] for i in range(0, b + neigh + 1)] for j in range(a - neigh, n)])
        elif b + neigh > m - 1: # on atteint le bord droit
            voisin = np.array([[world_without_ind[j, i] for i in range(b - neigh, m)] for j in range(a - neigh, n)])
    return voisin

def spatial_relocation (map, a, b, c, d): #map : List[List[int]], a : int(ligne), b : int(colonne), c : int(ligne), d : int(colonne)
    """Préconditions : 0 <= a <= n-1  and 0 <= b <= m-1  and 0 <= c <= n-1  and 0 <= d <= m-1 and n >= 2*neigh and m >= 2*neigh and map[c,d] == 0
    Renvoie le monde dans lequel l'individu de coordonnées [a, b] a été déplacé vers la case vide de coordonnées [c, d]"""
    ant = map[a, b] #on enregistre la cellule à déplacer (une fourmi)
    if map[c, d] == " ": #case vide
        # on supprime l'indidu du monde
        map_without_ant = np.delete(map, m*a + b) #on copie le monde sans l'individu
        map_without_ant = np.insert(map_without_ant, m*a + b, " ") #on remplace la place de l'individu par une case vide (avec un " ")
        map_without_ant = np.array([[map_without_ant[m*j + i] for i in range(0, m)] for j in range(0, n) ]) #on remet le monde sous forme de tableau
        # on remplace la case vide par l'individu
        relocation = np.delete(map_without_ant, m*c + d) #on copie le monde sans la case vide
        relocation = np.insert(relocation, m*c + d, ant) #on remplace la place de la case vide par l'individu (ant)
        relocation = np.array([[relocation[m*j + i] for i in range(0, m)] for j in range(0, n) ]) #on remet le monde sous forme de tableau
        return relocation

def ants (map, nb_ants):
    """Precondition : nb_ants >= 0
    Fait apparaitre nb_ants fourmis autour de la fourmilière"""
    ants_map = map
    for k in range(0, n):
        for l in range(0, m):
            if ants_map[k, l] == " ": #on vérifie que la case est vide
                for o in spatial_neighborhood(map, k, l, neigh):
                    for p in o:
                        if p == "h" and nb_ants > 0 : #on regarde si il y a une case fourmilière dans le voisinage
                            if ants_map[k, l] == " ": #on vérifie que la case est vide
                                ants_map = np.delete(ants_map, m*k + l) #on copie le monde en enlevant la case ayant la fourmilière pour voisin
                                ants_map = np.insert(ants_map, m*k + l, "f") #on remplace les cases initialement vide (avec un 0) par une case "fourmi" (avec un 2)
                                ants_map = np.array([[ants_map[m*j + i] for i in range(0, m)] for j in range(0, n) ])
                                nb_ants = nb_ants - 1
    q = 0                            
    while nb_ants > 0: #la fourmilière est entourée de fourmis
        q = q + 1
        for k in range(0, n):
            for l in range(0, m):
                if ants_map[k, l] == " ": #on vérifie que la case est vide
                    for o in spatial_neighborhood(map, k, l, neigh + q):
                        for p in o:
                            if p == "h" and nb_ants > 0 : #on regarde si il y a une case fourmilière dans le voisinage + q
                                if ants_map[k, l] == " ": #on vérifie que la case est vide
                                    ants_map = np.delete(ants_map, m*k + l) #on copie le monde en enlevant la case ayant la fourmilière pour voisin
                                    ants_map = np.insert(ants_map, m*k + l, "f") #on remplace les cases initialement vide (avec un " ") par une case "fourmi" (avec un f)
                                    ants_map = np.array([[ants_map[m*j + i] for i in range(0, m)] for j in range(0, n) ])
                                    nb_ants = nb_ants - 1                        
    return ants_map


def no_food(map, a, b):
    """Précondition : 0 >= a >= n, 0 >= b >= m
    Renvoie True si il n'y a pas de nourriture dans le voisinage de la case et False sinon."""
    for i in spatial_neighborhood(map, a, b, 1):
        for j in i:
            if j != " " and j != "h" and j != "f" and j != "s": #il y a de la nourriture dans le voisinage de la case, on peut utiliser le codage des caractères
                return False
    return True

def take_food(map, a, b):
    """Précondition : 0 >= a >= n, 0 >= b >= m
    Renvoie la même carte avec une unité de moins sur la nourriture"""
    food_map = map
    if food_map[a, b] == "f" or food_map[a, b] == "s":
        for i in range(0, 2):
            for j in range(0, 2):
                if spatial_neighborhood(map, a, b, 1)[i, j] != " " and spatial_neighborhood(map, a, b, 1)[i, j] != "h" and spatial_neighborhood(map, a, b, 1)[i, j] != "f": #la case [i, j] a de la nourriture
                    food_map = np.delete(food_map, m*(a - 1 + i) + (b - 1 + j)) #on copie le monde sans la case nourriture
                    food_map = np.insert(food_map, *(a - 1 + i) + (b - 1 + j), str(int(spatial_neighborhood(map, a, b, 1)[i, j] - 1))) #on enlève 1 à la valeur de nourriture
                    food_map = np.array([[food_map[m*k + l] for l in range(0, m)] for k in range(0, n) ]) #on remet le monde sous forme de tableau
        # on remplace la case vide par l'individu
    return food_map
    

def no_home(map, a, b):
    """Précondition : 0 >= a >= n, 0 >= b >= m
    Renvoie True si il n'y a pas de case fourmilière dans le voisinage et False sinon."""
    for i in spatial_neighborhood(map, a, b, 1):
        for j in i:
            if j == "h": #il n'y a pas de case fourmilière à côté
                return False
    return True

print(no_home(map, 1, 9))

def one_food_quest (map, a, b):
    """Précondition : 0 >= a >= n, 0 >= b >= m
    Si la case [a, b] est une fourmi, renvoie la matrice après son déplacement aléatoire sans retourner sur ses pas
    sinon renvoie la matrice initiale"""
    compt = 20
    move_map = map
    move_line = a #ligne de la fourmi
    move_column = b #colonne de la fourmi
    previous_location = - 10 #on retient la position précédente pour ne pas y revenir
    if move_map[a, b] == "f" : #la case sélectionnée est une fourmi
        while no_food(move_map, move_line, move_column) and compt > 0: #il n'y a pas de nourriture à coté de la fourmi            
            next_move = random.randint(0, 3) #on décide le procahin mouvement au hasard
            if next_move == 0 and previous_location != 2:
                if move_line > 0 and move_map[move_line - 1, move_column] == " ": #la prochaine case existe et est vide
                    move_map = spatial_relocation(move_map, move_line, move_column, move_line - 1, move_column) #déplacement vers le haut
                    previous_location = next_move
                    move_line = move_line - 1
            if next_move == 1 and previous_location != 3:
                if move_column < m - 1 and move_map[move_line, move_column + 1] == " ":
                    move_map = spatial_relocation(move_map, move_line, move_column, move_line, move_column + 1) #déplacement vers la droite
                    previous_location = next_move                  
                    move_column = move_column + 1
            if next_move == 2 and previous_location != 0:
                if move_line < n - 1 and move_map[move_line + 1, move_column] == " ":
                    move_map = spatial_relocation(move_map, move_line, move_column, move_line + 1, move_column) #déplacement vers le bas
                    previous_location = next_move
                    move_line = move_line + 1
            if next_move == 3 and previous_location != 1:
                if move_column > 0 and move_map[move_line, move_column - 1] == " ":
                    move_map = spatial_relocation(move_map, move_line, move_column, move_line, move_column - 1) #déplacement vers la gauche
                    previous_location = next_move
                    move_column = move_column - 1
            print(no_food(move_map, move_line, move_column), move_line, move_column)
            compt = compt - 1
            print(move_map)
    return move_map
            

def ants_food_quest(map):
    
    """Effectue le déplacement aléatoire de toutes les fourmis jusqu'à ce qu'il y en ait une qui trouve de la nourriture"""
    move_map = map
    move_line = - 1
    move_column = - 1
    food = False
    while food == False:
        for i in range(0, n):
            for j in range(0, m):
                if move_map[i, j] == "f" : #la case sélectionnée est une fourmi
                    next_move = random.randint(0, 3) #on décide le prochain mouvement au hasard
                    if next_move == 0:
                        if i > 0 and move_map[i - 1, j] == " ": #la prochaine case existe et est vide
                            move_map = spatial_relocation(move_map, i, j, i - 1, j) #déplacement vers le haut
                            move_line = i - 1
                            move_column = j
                    if next_move == 1:
                        if j < m - 1 and move_map[i, j + 1] == " ":
                            move_map = spatial_relocation(move_map, i, j, i, j + 1) #déplacement vers la droite
                            move_line = i
                            move_column = j + 1
                    if next_move == 2:
                        if i < n - 1 and move_map[i + 1, j] == " ":
                            move_map = spatial_relocation(move_map, i, j, i + 1, j) #déplacement vers le bas
                            move_line = i + 1
                            move_column = j
                    if next_move == 3:
                        if j > 0 and move_map[i, j - 1] == " ":
                            move_map = spatial_relocation(move_map, i, j, i, j - 1) #déplacement vers la gauche
                            move_line = i
                            move_column = j - 1
                    if no_food(move_map, move_line, move_column) == False:
                        food = True
        print(move_map)
        print(food)
    return move_map

#print(one_food_quest(ants(generate_simple_map(n, m), 1), 3, 3))
#print(ants_food_quest(ants(generate_simple_map(m, n), 10)))

def return_home (map):
    """Si une fourmi a trouvé de la nourriture, effectue le déplacement de celle-ci vers la fourmilière"""
    move_map = map
    line_found = - 1 #ligne de la fourmi découvreuse de nourriture
    column_found = - 1 #colonne de la fourmi découvreuse de nourriture
    eratic_move = 0
    next_move = 0
    for i in range(0, n):
        for j in range(0, m):
            if move_map[i, j] == "f" :
                if no_food(move_map, i, j) == False:
                    line_found = i
                    column_found = j
                    move_map = np.delete(move_map, m*line_found + column_found) #on copie le monde sans l'individu
                    move_map = np.insert(move_map, m*line_found + column_found, "s") #on remplace la fourmi par une fourmi ayant de la nourriture (avec un "s")
                    move_map = np.array([[move_map[m*k + l] for l in range(0, m)] for k in range(0, n) ]) #on remet le monde sous forme de tableau
                   # move_map = take_food(move_map, line_found, column_found)
                    while no_home(move_map, line_found, column_found):
                        print(move_map)
                        # mouvement diagonal vers la fourmilière
                        if line_found >= n/2 and column_found >= m/2 and move_map[line_found - 1, column_found - 1] == " ": #la fourmi est dans le quart bas droit de la map
                            move_map = spatial_relocation(move_map, line_found, column_found, line_found - 1, column_found - 1)
                            line_found = line_found - 1
                            column_found = column_found - 1
                        if line_found >= n/2 and column_found <= m/2 and move_map[line_found - 1, column_found + 1] == " ": #la fourmi est dans le quart bas gauche de la map
                            move_map = spatial_relocation(move_map, line_found, column_found, line_found - 1, column_found + 1)
                            line_found = line_found - 1
                            column_found = column_found + 1  
                        if line_found <= n/2 and column_found <= m/2 and move_map[line_found + 1, column_found + 1] == " ": #la fourmi est dans le quart haut gauche de la map
                            move_map = spatial_relocation(move_map, line_found, column_found, line_found + 1, column_found + 1)
                            line_found = line_found + 1
                            column_found = column_found + 1   
                        if line_found <= n/2 and column_found >= m/2 and move_map[line_found + 1, column_found - 1] == " ": #la fourmi est dans le quart haut droit de la map
                            move_map = spatial_relocation(move_map, line_found, column_found, line_found + 1, column_found - 1)
                            line_found = line_found + 1
                            column_found = column_found - 1   
                        eratic_move = eratic_move + 1
                    
                        # mouvement eratique périodique
                        if eratic_move > 2:
                            next_move = random.randint(0, 3) #on décide le procahin mouvement au hasard
                            if next_move == 0:
                                if line_found > 0 and move_map[line_found - 1, column_found] == " ": #la prochaine case existe et est vide
                                    move_map = spatial_relocation(move_map, line_found, column_found, line_found - 1, column_found) #déplacement vers le haut
                                    line_found = line_found - 1
                            if next_move == 1:
                                if column_found < m - 1 and move_map[line_found, column_found + 1] == " ":
                                    move_map = spatial_relocation(move_map, line_found, column_found, line_found, column_found + 1) #déplacement vers la droite
                                    column_found = column_found + 1
                            if next_move == 2:
                                if line_found < n - 1 and move_map[line_found + 1, column_found] == " ":
                                    move_map = spatial_relocation(move_map, line_found, column_found, line_found + 1, column_found) #déplacement vers le bas
                                    line_found = line_found + 1
                            if next_move == 3:
                                if column_found > 0 and move_map[line_found, column_found - 1] == " ":
                                    move_map = spatial_relocation(move_map, line_found, column_found, line_found, column_found - 1) #déplacement vers la gauche
                                    column_found = column_found - 1
                            print(move_map)
                            eratic_move = 0
                        if no_home(move_map, line_found, column_found) == False:
                            move_map = np.delete(move_map, m*line_found + line_found) #on copie le monde sans l'individu
                            move_map = np.insert(move_map, m*line_found + column_found, "f") #on remplace la fourmi par une fourmi sans nourriture (avec un "f")
                            move_map = np.array([[move_map[m*k + l] for l in range(0, m)] for k in range(0, n) ]) #on remet le monde sous forme de tableau
    return move_map

print(return_home(ants_food_quest(ants(map, 10))))
                        

186
[[' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' '1' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'h' 'h' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'h' 'h' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' '